# Защита персональных данных клиентов страховой компании


Цели и задачи проекта: 
- Нам нужно защитить данные клиентов страховой компании «Хоть потоп». 
- Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, обосновать корректность его работы.
- Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

Набор данных находится в файле /datasets/insurance.csv. 


## Загрузка данных

In [ ]:
#загрузим потенциально необходимые библиотеки, пакеты и пр.
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:,.2f}".format)

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv('/datasets/insurance.csv')
data.head()

In [ ]:
data.head()

Проанализируем полученную информацию

In [ ]:
data.info()

In [ ]:
data.describe()

Зависимость целевого значения от других признаков:

In [ ]:
sns.set_style('ticks')
sns.pairplot(data, hue='Страховые выплаты', height=2.5, diag_kind='hist',palette='deep')
plt.suptitle("Графики зависимости целевой метки от параметров в данных", y=1.02, fontsize=13)
plt.show()

In [ ]:
corr = data.corr()
sns.heatmap(corr)

In [ ]:
data = data.rename(
    columns={
    'Страховые выплаты':'insurance_payments'
}
  )

Датасет почти не нуждался в предобработке. Из графиков видно: по объёму не выплат страховок больше всего, а если и производятся, то чаще всего распространяются на людей старше 40 лет с семьями от двух человек.
- В датафрейме 5000 строк и 5 столбцов.
- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$


### Разобьём данные на обучающую и целевую выборки

In [ ]:
features = data.drop('insurance_payments', axis=1)
target = data['insurance_payments']

In [ ]:
# Напишем рекурсивную функцию, выдающую на выходе обратимую матрицу 4х4
def invert():
    s = np.random.normal(size=(4, 4))
    try:
        np.linalg.inv(s)
        return s
    except:
        invert()

### Используя класс, воспроизведём линейную регрессию

In [ ]:
class LinearRegression:
    def fit(self, features, target):
        X = np.concatenate((np.ones((len(features), 1)), features), axis=1)
        y = target
        w = np.linalg.inv(X.T @ X) @ X.T @ y
        self.w = w[1:]
        self.w0 = w[0]
        
    def predict(self, test_features):
        return test_features @ self.w + self.w0

### Обучим модель и найдём показатель R2 до преобразования

In [ ]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
R2_before = r2_score(target, predictions)
print('Показатель R2 до преобразования равен:', R2_before)

### Обучим модель и найдём показатель R2 после преобразования

In [ ]:
# Умножим фичи на обратимую матрицу
new_features = features @ invert()

# Повторим поиск параметра R2 на новых фичах
model = LinearRegression()
model.fit(new_features, target)
predictions = model.predict(new_features)
R2_after = r2_score(target, predictions)
print('Показатель R2 после преобразования равен:', R2_after)

### Напишем итоговый ответ на задание 

 **Мы проверили качество обучения фичей до преобразованиz и после преобразования (умноженную на обратимую матрицу, полученную рандомно) через коэффициент детерминации R2. Качество модели осталось на том же уровне.**

**Обоснование:** **Обоснуем отсутствие изменений в качестве линейной регрессии после её преобразования (умножения на обратимую матрицу), где**
- $a$ — вектор предсказаний

- $a1$ — вектор предсказаний после умножения на матрицу

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $X1$ — матрица признаков после умножения (нулевой столбец состоит из единиц)

- $Е$ — единичная матрица

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Вектор предсказаний линейной регрессии имеет вид $$a = Xw$$ 

Докажем, что после умножения на матрицу Р, он не будет иметь значительных изменений $$a1 = а$$

Формула Х1:  
$$ X1 = X P $$  

Формула веса :  
$$ w = (X^T X)^{-1} X^T y $$
Подставим X1 в формулу веса после умножения и выполним преобразования:  
$$ w1 = ((XP)^T XP)^{-1} (XP)^T y  => $$
$$ w1 = (P^T (X^T X) P)^{-1} P^T X^T y => $$
$$ w1 = P^{-1} (X^T X)^{-1} (P^T)^{-1} P^T X^T y => $$
$$ w1 = P^{-1} ((X^T X)^{-1}) E X^T y $$
$$ w1 = P^{-1} w $$

Подставим w1 в формулу вектора предсказаний линейной регрессии после умножения и выполним преобразования:

$$ a1 = X1 w1 $$
$$ a1 = X P P^{-1} w $$
$$ a1 = X w $$
Таким образом, мы можем сделать вывод, что $$a=a1$$
**Что и требовалось доказать**

## Алгоритм преобразования

### Предложить вариант преобразований данных для задачи и обосновать: почему не поменяется качество предсказаний


*Можно выполнить преобразования методами StandartScaler или MinMaxScaler*

*В качестве примера рассмотрим StandartScaler*

In [ ]:
# Создадим копию нашей фичи перед преобразованием
features_scaler = features.copy()

# Создадим список из столбцов наших фичей
numeric = list(features.columns)

# Выполним преобразования методом StandartScaler
scaler = StandardScaler()
scaler.fit(features_scaler[numeric])
features_scaler[numeric] = scaler.transform(features_scaler[numeric])

In [ ]:
# Обучим модель после преобразования
model = LinearRegression()
model.fit(features_scaler, target)
predictions = model.predict(features_scaler)
R2_scaler = r2_score(target, predictions)
print('Показатель R2 для метода StandartScaler равен:', R2_scaler)

Формула StandartScaler имеет вид $$z = (x - u) / s$$, где
- $х$ — изменяемый признак

- $u$ — среднее значение этого признака

- $s$ — стандартное отклонение

### Создадим функцию, которая воспроизведёт StandartScaler

In [ ]:
def standart_scaller(X):
    u = np.mean(X)
    s = np.std(X)
    new = []
    for i in range(len(X)):  
        vector=[]
        for j in range(len(X.columns)):  
            vector.append(
                (np.array(X)[i,j] - u[j]) / s[j])
        new.append(np.nan_to_num(vector))
    new = np.array(new)
    return new

In [ ]:
# Вытащим преобразованный массив
features_scaler_new = standart_scaller(features)
# Обучим модель после преобразования
model = LinearRegression()
model.fit(features_scaler_new, target)
predictions = model.predict(features_scaler_new)
R2_scaler_new = r2_score(target, predictions)
print('Показатель R2 для метода StandartScaler равен:', R2_scaler_new)

### Сравним массивы после стандартного StandartScaler и после нашего ручного в виде функции

In [ ]:
# Наш созданный StandartScaler из функции
features_scaler_new

In [ ]:
# Стандартный StandartScaler
np.array(features_scaler)

*Они идентичны*

## Проверка алгоритма

### Сравним показатели R2 до умножения на обратимую матрицу и после

In [ ]:
R2_before, R2_after

### Сравним показатели R2 до преобразования StandartScaler и после

Cравнивать показатели R2 между стандартным StandartScaler и StandartScaler из ручной функции не имеет смысла, мы уже убедились, что массивы полностью идентичны,следовательно будут идентичны и все остальные показатели. Так что для сравнения можно взять любой из них

In [ ]:
R2_before, R2_scaler_new

В обоих случаях показатели равны

- Мы загрузили файл с данными и сделали предобработку данных. 
- Затем ответили на вопрос: изменится ли качество линейной регрессии при умножении её на обратимую матрицу. 
- Мы увидели, что в показатель коэффиент детерминации остался на том же уровне. 
- Следом мы обосновали наш вывод путём преобразования формул, что дало нам уже точный ответ, что качество регрессии останется на том же уровне. 
- Были предложены два способа преобразования фичей без потерь качества предсказаний:Это методы StandartScaler или MinMaxScaler. 
- Исследовательскую работу мы провели по StandartScaler. Изменили фичи стандартным способом и через функцию, чтобы можно было сравнить и убедиться, что мы не ошиблись в формуле, после чего полученный массив опять обучили и вытащили метрику R2.
- Она так же не повлияла на качество регрессии. С помощью преобразований мы не теряем качество, но при этом, если наши данные попадут не в те руки, то они окажутся бесполезными, следовательно можно сказать, что они у нас достаточно защищены.